In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

from src.construct import util

In [4]:
data_dir = "data/automatic/gas_storage.xlsx"
gas_storage_cap = pd.read_excel(data_dir, sheet_name="WGV by type TWh", skiprows=3, index_col=0)

In [5]:
all_sites = pd.read_excel(data_dir, sheet_name="Storage DB ", skiprows=1, index_col=0)

In [6]:
daily_energy_cap = (
    all_sites
    .loc[(all_sites.index == 'Serbia') | (all_sites["in EU28 SUM"] == "y"), ["Withdrawal\ntechnical\nGWh/day", "Injection\ntechnical\nGWh/day"]]
    .where(lambda x: x > 0)
    .mean(axis=1)
)
hourly_energy_cap = (daily_energy_cap / 24).groupby(level=0).sum()

In [15]:
gas_storage_data = pd.concat(
    [hourly_energy_cap * 1e3, # Scale GW to MW
     gas_storage_cap.reindex(hourly_energy_cap.index).Total * 1e6],  # scale TWh to MWh
    keys=['energy_cap_gas_storage_mw', 'storage_cap_gas_storage_mwh'],
    axis=1
).index.map(util.get_alpha3).rename('country_code')

In [16]:
gas_storage_data

Index(['AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'DNK', 'FRA', 'DEU', 'GRC', 'HUN',
       'ITA', 'LVA', 'NLD', 'POL', 'PRT', 'ROU', 'SRB', 'SVK', 'ESP', 'SWE',
       'GBR'],
      dtype='object', name='country_code')

In [106]:
units = gpd.read_file('build/units.geojson')

/Users/brynmorp/miniconda3/envs/euro-spores/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [107]:
gas_data = pd.concat([gas_storage_cap.reindex(hourly_energy_cap.index).Total, hourly_energy_cap], axis=1, keys=['storage_cap', 'energy_cap'])

/Users/brynmorp/miniconda3/envs/euro-spores/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [124]:
gas_data.index = gas_data.index.map(util.get_alpha3).rename("country_code")
bar = units.set_index(["id", "country_code"])
foo = gas_data.reindex(bar.index, level="country_code")#.set_index(units.set_index(["id", "country_code"]).index)
foo.div(foo.count(level='country_code')).droplevel('country_code')

/Users/brynmorp/miniconda3/envs/euro-spores/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,storage_cap,energy_cap
id,,
ALB_1,NaN,NaN
AUT_1,30.73950,13.073122
AUT_2,30.73950,13.073122
AUT_3,30.73950,13.073122
BEL_1,9.00130,5.367500
...,...,...
SVN_1,NaN,NaN
SWE_1,0.02425,0.082500
SWE_2,0.02425,0.082500
